# Clustering con scikit-learn

En este trabajo se realizan tareas de preprocesamiento y clustering sobre el dataset `fetch_20newsgroups` de `scikit-learn`. Se aplican y comparan dos algoritmos de clustering, a saber:

* `KMeans`
* `AgglomerativeClustering`

# Procesamiento sobre dataset original

## Carga del dataset

Se carga el dataset y se imprime información sobre el mismo.

In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = [
 'rec.motorcycles',
 'rec.sport.baseball',
 'sci.space',
]

df = fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'), categories=categories,
                             shuffle=True, random_state=42)

print(f'Se obtienen {len(df.data)} instancias, de acuerdo a lo solicitado.')
print('Las etiquetas de clase son : "' + str(set(df.target))[1:-1] + '"')

Se obtienen 2977 instancias, de acuerdo a lo solicitado.
Las etiquetas de clase son : "0, 1, 2"


In [2]:
print(df.DESCR[:1085])

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality 

El dataset contiene una colección de noticias sobre 20 tópicos diferentes, conformando un total de 18846 instancias, divididas en un subset para train y otro para test (al especificar `subset='all'` se obtienen los datos de ambos).

A pesar de existir un extractor adicional, `fetch_20newsgroups_vectorized`, con la información de texto preprocesada, se obtienen los datos en una dimensión y le realizamos tareas de preprocesamiento.

In [3]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

df_data = pd.concat([pd.DataFrame(df.data,columns=['news']),pd.DataFrame(df.target,columns=['class'])], axis=1)
df_data

,news,class
0,Wednesday's game of Beloved Yakult Swallows\n\...,1
1,"\nOkay, I think we all agree that singles hitt...",1
2,"Having run completely out of time, I've got to...",1
3,"To Mr. Millitello -\n\n\tListen, Sammy, can yo...",1
4,"\n\nHey! I LIKE quiche, even if I did have to ...",0
...,...,...
2972,The Centaur is controlled technology..\nState ...,2
2973,,0
2974,\n\nWho says there is no mineral rights to be ...,2
2975,"\n [ etc. ]\n\nHey, he's the only manager so f...",1


Puede verse la existencia de instancias vacías (sin contenido de texto), a continuación se obtiene información sobre la longitud, en caracteres, de las noticias.

In [4]:
df_data.loc[:,'news'].str.len().describe().round(2)

count     2977.00
mean       835.46
std       2151.86
min          0.00
25%        195.00
50%        411.00
75%        834.00
max      56793.00
Name: news, dtype: float64

## Preprocesamiento de texto

Por lo visto, el primer paso en el preprocesamiento es el de eliminar las instancias que no contengan información relevante. Se determina conservar las que tengan más de 10 caracteres. A continuación, se genera un `DataFrame`, separando luego el mismo en sus etiquetas de clase (`labels`), y el contenido de texto (`news`).

In [5]:
df_data = df_data[df_data.loc[:,'news'].str.len() > 10]
print(len(df_data))

labels = df_data['class']
dataset = df_data['news']

2875


In [6]:
df_data.groupby('class').count()

,news
class,
0,965
1,955
2,955


Se eliminan así aproximadamente 100 instancias, adicionalmente se ve que la cantidad de instancias por clase es similar.

Luego de realizar pruebas sobre el texto se determinan finalmente la aplicación de las siguientes tareas:

* Eliminación de instancias con menos de 10 caracteres.
* Conservación únicamente de tokens alfabéticos.
* Remoción de stop words.
* Stemming.
* Inclusión de n-grams (hasta 2 palabras).
* Control de frecuencia mínima (5 documentos).
* Generación de matriz TF/IDF.

Principalmente, se busca conservar la información más relevante de cada instancia, lo que se logra a través de la remoción de las stop words y el proceso de stemming. Con la generación de la matriz TF/IDF le asignamos un peso a cada token y para cada instancia, relacionado a su importancia relativa en la misma. La eliminación de los tokens que contienen números, se determina luego de un primer análisis.

Para determinar la cantidad de n-gramas a incluir, y el control de frecuencia mínima, se realiza un clustering de ejemplo, variando los dos primeros parámetros y evaluando las métricas de acuerdo a un criterio externo. Para esto se utiliza un algoritmo jerárquico (`AgglomerativeClustering`), colocando como único parámetro la cantidad de clusters a encontrar (en este caso 3, la misma cantidad de etiquetas de clase), y utilizando el resto por defecto. Luego de ejecutar algunos ciclos `for`, se observa que se obtiene un mejor valor de `v-measure` conservando los tokens que se mantienen en al menos 5 documentos, e incluyendo n-gramas de hasta 2 palabras.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, \
ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer

# Creamos el stemmer
stemmer = PorterStemmer()

# Construimos un tokenizer
tokenizer = CountVectorizer().build_tokenizer()

# Obtenemos las stopwords que provee scikit-learn
stop_words = ENGLISH_STOP_WORDS

# Definimos una función que aplica el stemming luego de tokenizar y remover stopwords
def  stem_tokenizer(doc):
    # Aplica la tokenización
    tokens = tokenizer(doc)
    # Retorna la lista de tokens luego de aplicar stemming si no es un stopword
    return list(stemmer.stem(w) for w in tokens if w not in stop_words and w.isalpha()) 

# Aplicamos el procesamiento
vectorizer = TfidfVectorizer(tokenizer=stem_tokenizer, ngram_range=(1,2), min_df=5)
tokens = vectorizer.fit_transform(dataset.tolist())
features = vectorizer.get_feature_names()

df_tokens = pd.DataFrame(tokens.toarray(), columns=features)
print(f'Son generados para el dataset, {df_tokens.shape[1]} tokens únicos')

Son generados para el dataset, 5912 tokens únicos


In [8]:
df_tokens.columns[420:440]

Index(['bastard', 'bat', 'bat averag', 'bat eye', 'bats', 'batter',
       'batter box', 'batteri', 'battl', 'bay', 'bay area', 'baylor', 'bb',
       'bb access', 'bb updat', 'bc', 'bc canada', 'bchm', 'bchm biochem',
       'bdi'],
      dtype='object')

Finalmente, se eliminan las filas que no muestren valor para ningún token, estas intancias no se consideran representativas de las clases. Este proceso ayuda a evitar errores posteriores con los algoritmos de clustering.

Se realiza el mismo paso sobre el `DataFrame` original, principalmente para afectar de igual manera a las etiquetas de clase.

In [9]:
mask = (df_tokens != 0).any(axis=1)

print(f'Se eliminan {len(df_tokens[mask == False].index)} instancias en este último paso.')

df_data = df_data.drop(df_tokens[mask == False].index)
df_tokens = df_tokens.drop(df_tokens[mask == False].index)

labels = df_data['class']
dataset = df_data['news']

Se eliminan 12 instancias en este último paso.


## Clustering

En este caso, la finalidad de aplicar tareas de clustering es el de buscar un agrupamiento de los datos que logre asemejarse al ground truth. Por esto, se evaluarán los algoritmos de clustering, principalmente, por la capacidad que tengan de descubrir estas clases implícitas (criterio externo).

Los algoritmos a utilizar son el `KMeans` y `AgglomerativeClustering` de `scikit-learn`, de K-means y clustering jerárquico aglomerativo del tipo bottom-up.

In [0]:
from sklearn.cluster import KMeans, AgglomerativeClustering

km = KMeans(n_clusters=3, 
            init='k-means++', 

            max_iter=300,
            tol=1e-04,
            random_state=0)

ac = AgglomerativeClustering(n_clusters=3)

En el algoritmos de K-means, los parámetros utilizados son:

* **k = 3**. De manera de generar la misma cantidad de clusters que las clases implícitas.
* **`init = 'k-means++'`**. Se busca una mejor elección de las semillas en la inicialización del algoritmo de clustering.
* **tol = $1.10^{-4}$**, **max_iter = 300**. Estos determinan la cantidad de ciclos que se ejecutarán. La tolerancia determina el criterio de terminación, respecto a los cambios registrados en el SSE (suma de errores cuadráticos). De no satisfacerse esta condición, se realiza una iteración de 300 ciclos.

Para el algoritmo `AgglomerativeClustering`, se indica la cantidad de clusters a encontrar, siendo **`n_clusters = 3`**. Por defecto, la función de similitud es utilizada es **`linkage = 'ward'`**, que minimiza la varianza de los grupos a unir. Se evaluarán diferentes funciones, a modo de probar su eficacia en este problema. En ambos se utiliza la distancia Euclideana (se intentó computar la similitud de coseno y utilizar otras funciones de similitud, pero no se obtuvieron resultados satisfactorios).

## Evaluación de los resultados

Principalmente, cabe evaluar los modelos a través de un criterio externo. De esta manera se puede saber su capacidad de agrupar en lso clusters a elementos pertenecientes a la misma clase implícita. 

A continuación, se realizan predicciones y se las compara con el ground truth, a través de las cuatro métricas salientes bajo este criterio: **Homogeneity**, **Completeness**, **V-Measure** y **Rand Index**.

In [11]:
from sklearn import metrics

dict_cl = {km: 'K-Means', ac: 'Jerárquico Aglomerativo'}

for key, value in dict_cl.items():
  labels_pred = key.fit_predict(df_tokens.to_numpy())
  h, c, v = metrics.homogeneity_completeness_v_measure(labels, labels_pred)
  print(value + ':')
  print('Homogeneity: %.3f' %h)
  print('Completeness: %.3f' %c)
  print('V-Measure: %.3f' %v)
  print('Adjusted Rand-Index: %.3f'
      % metrics.adjusted_rand_score(labels, labels_pred))
  print('\n')

K-Means:
Homogeneity: 0.291
Completeness: 0.330
V-Measure: 0.309
Adjusted Rand-Index: 0.249


Jerárquico Aglomerativo:
Homogeneity: 0.251
Completeness: 0.279
V-Measure: 0.264
Adjusted Rand-Index: 0.224




# Procesamiento sobre dataset filtrado

A continuación, se repite el procesamiento y análisis, llevado a cabo con el dataset anterior, a uno de similares características pero previamente filtrado, con lo que se tienen diferentes instancias.


## Carga del dataset

In [13]:
from google.colab import files
import io

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'Se subió el archivo {fn} con longitud {len(uploaded[fn])}')
  df_news = pd.read_csv(io.StringIO(uploaded['news-filtered.csv'].decode('utf-8')), sep=',')

print(f'Se cargaron {df_news.shape[0]} instancias')
df_news.head()

Saving news-filtered.csv to news-filtered (6).csv
Se subió el archivo news-filtered.csv con longitud 2028466
Se cargaron 1692 instancias


,Unnamed: 0,class,news
0,2249,1,I thought I'd post my predicted standings sinc...
1,2748,2,Archive-name: space/acronyms\nEdition: 8\n\nAc...
2,1144,1,Philadelphia at Chicago: Teams tied for 1st a...
3,1509,2,Archive-name: space/new_probes\nLast-modified:...
4,2,1,"Having run completely out of time, I've got to..."


In [14]:
df_news.groupby('class').count().loc[:, 'news']

class
0    474
1    607
2    611
Name: news, dtype: int64

In [15]:
df_news.loc[:,'news'].str.len().describe().round(2)

count     1692.00
mean      1187.40
std       2644.30
min         24.00
25%        340.75
50%        620.50
75%       1171.25
max      56793.00
Name: news, dtype: float64

Puede verse que se mantienen las etiquetas de clase, en este caso teniendo menos instancias en la primera de ellas. A diferencia del primer datset no se tienen instancias sin contenido de texto, registrando 24 caracteres como mínimo.

In [0]:
labels = df_news['class']
dataset = df_news['news']

## Preprocesamiento de texto

Inicialmente, se realiza el mismo procesamiento que el previamente aplicado.

In [17]:
vectorizer = TfidfVectorizer(tokenizer=stem_tokenizer, ngram_range=(1,2), min_df=5)
tokens = vectorizer.fit_transform(dataset.tolist())
features = vectorizer.get_feature_names()

df_tokens = pd.DataFrame(tokens.toarray(), columns=features)
print(f'Son generados para el dataset, {df_tokens.shape[1]} tokens únicos')

Son generados para el dataset, 4871 tokens únicos


In [18]:
mask = (df_tokens != 0).any(axis=1)
len(df_tokens[mask == False].index)

0

En este caso, todas las instancias quedan representadas al menos por un token, con lo que ninguna será descartada.

Se procede a realizar el clustering sobre la información de texto pre-procesada. En este caso se muestran los resultados para los algoritmos de K-Means y Jerárquico previamente introducidos, agregando además los resultados de este último con diferentes parámetros.

Se evalúan las mismas métricas que para el dataset original.

## Clustering y evaluación de los resultados

In [19]:
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn import metrics

km = KMeans(n_clusters=3, 
            init='k-means++', 

            max_iter=300,
            tol=1e-04,
            random_state=0)

ac = AgglomerativeClustering(n_clusters=3, linkage='ward')
ac_cos_single = AgglomerativeClustering(n_clusters=3, affinity='cosine', linkage='single')
ac_single = AgglomerativeClustering(n_clusters=3, linkage='single')
ac_complete = AgglomerativeClustering(n_clusters=3, linkage='complete')
ac_gavg = AgglomerativeClustering(n_clusters=3, linkage='average')

dict_cl = {km: 'K-Means', ac: "Jerárquico, Ward", ac_single: "Jerárquico, Single link", 
           ac_cos_single: "Jerárquico, Single link, similitud coseno", 
           ac_complete: "Jerárquico, Complete link", ac_gavg: "Jerárquico, Group average"}

for key, value in dict_cl.items():
  labels_pred = key.fit_predict(df_tokens.to_numpy())
  h, c, v = metrics.homogeneity_completeness_v_measure(labels, labels_pred)
  print(value + ':')
  print('Homogeneity: %.3f' %h)
  print('Completeness: %.3f' %c)
  print('V-Measure: %.3f' %v)
  print('Adjusted Rand-Index: %.3f'
      % metrics.adjusted_rand_score(labels, labels_pred))
  print('\n')

K-Means:
Homogeneity: 0.970
Completeness: 0.971
V-Measure: 0.970
Adjusted Rand-Index: 0.983


Jerárquico, Ward:
Homogeneity: 0.815
Completeness: 0.816
V-Measure: 0.815
Adjusted Rand-Index: 0.879


Jerárquico, Single link:
Homogeneity: 0.001
Completeness: 0.136
V-Measure: 0.002
Adjusted Rand-Index: 0.000


Jerárquico, Single link, similitud coseno:
Homogeneity: 0.001
Completeness: 0.136
V-Measure: 0.002
Adjusted Rand-Index: 0.000


Jerárquico, Complete link:
Homogeneity: 0.074
Completeness: 0.079
V-Measure: 0.076
Adjusted Rand-Index: 0.057


Jerárquico, Group average:
Homogeneity: 0.006
Completeness: 0.164
V-Measure: 0.011
Adjusted Rand-Index: 0.002




Como puede verse, los mejores resultados son obtenidos para el algoritmo K-Means. Dentro de los jerárquicos ejecutados, se observa un buen rendimiento únicamente para el que utiliza la función de similitud de Ward, registrando rendimientos pobres en el resto de los casos.

Finalmente, tal como fuera probado para el dataset original (el código no está incluído previamente), se analiza el rendimiento variando en la etapa de pre-procesamiento de texto el control de frecuencia mínima.

In [21]:
import numpy as np

for i in np.arange(3)+2:
  vectorizer = TfidfVectorizer(tokenizer=stem_tokenizer, ngram_range=(1,2), min_df=i)
  tokens = vectorizer.fit_transform(dataset.tolist())

  print(f'{tokens.shape[1]} tokens generados, siendo la frecuencia mínima de {i} documentos.\n')

  labels_a = km.fit_predict(tokens.toarray())
  labels_b = ac.fit_predict(tokens.toarray())
  h_a, c_a, v_a = metrics.homogeneity_completeness_v_measure(labels, labels_a)
  h_b, c_b, v_b = metrics.homogeneity_completeness_v_measure(labels, labels_b)

  print('K-Means')
  print('V-Measure: %.3f' %metrics.v_measure_score(labels, labels_a))
  print('Adjusted Rand-Index: %.3f'
      % metrics.adjusted_rand_score(labels, labels_a))
  print('\nJerárquico Aglomerativo:')
  print('V-Measure: %.3f' %metrics.v_measure_score(labels, labels_b))
  print('Adjusted Rand-Index b: %.3f'
      % metrics.adjusted_rand_score(labels, labels_b))
  print('\n')

21128 tokens generados, siendo la frecuencia mínima de 2 documentos.

K-Means
V-Measure: 0.963
Adjusted Rand-Index: 0.980

Jerárquico Aglomerativo:
V-Measure: 0.826
Adjusted Rand-Index b: 0.885


9643 tokens generados, siendo la frecuencia mínima de 3 documentos.

K-Means
V-Measure: 0.953
Adjusted Rand-Index: 0.973

Jerárquico Aglomerativo:
V-Measure: 0.865
Adjusted Rand-Index b: 0.916


6342 tokens generados, siendo la frecuencia mínima de 4 documentos.

K-Means
V-Measure: 0.993
Adjusted Rand-Index: 0.997

Jerárquico Aglomerativo:
V-Measure: 0.832
Adjusted Rand-Index b: 0.891




Similar a lo que ocurre previamente, los mejores resultados se obtienen, para cada uno de los casos, con el algoritmo K-Means.


# Conclusiones

Se seleccionan dos algortimos específicos, mostrando el de K-Means un mejor rendimiento en la totalidad de los criterios externos evaluados. Se realizaron además diversas pruebas sobre el algoritmo jerárquico, mediante las cuales resultó como mejor alternativa, la que utiliza la función de similitud de Ward, computando las distancias Euclidianas.

Sobre el preprocesamiento de texto, el mejor resultado con el algoritmo jerárquico se alcanza para una frecuencia mínima de 3 documentos. El mejor valor alcanzado es para el clustering con K-Means, conservando tokens que aparecen en por lo menos 4 documentos. Con este último las métricas V-Measure y Adjusted Rand Index son aproximadamente 1, con lo que se obtiene una efectividad en encontrar el ground truth es casi óptima.

Por esto, se recomienda la utilización del algoritmo K-Means para el clustering del dataset obtenido. Un uso que puede darse es el de separar noticias (o documentos en general) por tópico. De no conocerse el ground truth, es decir, el tópico real, se espera un buen agrupamiento de las noticias de acuerdo a su similaridad en el contenido. Para este tipo de problemas, se recomienda variar la cantidad de clusters, de manera de obtener buenos resultados al evaluar mediante un criterio interno.